## Amharic Named Entity Recognition (NER) system

### Importing the necessary libraries

In [1]:
import os
import sys

import pandas as pd
import demoji  
import regex as re

# from amseg.amharicNormalizer import AmharicNormalizer as normalizer
# from amseg.amharicSegmenter import AmharicSegmenter



In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
sys.path.append(os.path.abspath('../scripts'))
from Utils import DataUtils

utils = DataUtils()

#### Loading and cleaning data 

In [4]:
telegram_data = pd.read_csv('../data/telegram_data.csv')

In [5]:
data_clean_message = telegram_data[~(telegram_data['Message'].isna())]
data_clean_message.loc[:,'message_no_emoji'] = data_clean_message['Message'].apply(lambda x: utils.remove_emoji(x))
data_clean_message = data_clean_message[data_clean_message['message_no_emoji'].str.contains("Price")]

In [6]:
replacements = {
    'Price': 'ዋጋ',
    'Br' : ' ብር',
    'br ': ' ብር',
    'Address': ' አድራሻ',
    'Addis Ababa': 'አዲስ አበባ',
    'HayaHulet': 'ሀያሁለት', 
}

data_clean_message['message_no_emoji'] = data_clean_message['message_no_emoji'].replace(replacements)


In [279]:
message_only = data_clean_message['message_no_emoji']
message_only.to_csv('../data/message.csv', index=False)

#### Preprocess text data

In [275]:
messages = pd.read_csv('../data/message.csv')
messages['normalized'] = messages['message_no_emoji'].apply(lambda x: normalizer.normalize(x))

In [ ]:
corpus = []
for i in range(messages.shape[0]):
  corpus.extend(utils.tokenizer(messages['normalized'].iloc[i]))

In [ ]:
labeled_data = utils.label_conll_format(corpus)


co_nll_output = []
for word, label in labeled_data:
    co_nll_output.append(f"{word} {label}")

co_nll_formatted_output = "\n".join(co_nll_output) + "\n"

with open('../data/conll_output.txt', 'w', encoding='utf-8') as f:
    f.write(co_nll_formatted_output)


#### Conll Format

In [1]:
with open('../data/conll_output.txt', mode='r', encoding='utf-8') as file:
    counter = 0
    for line in file:
        if counter == 15:
            break
        print(line)
        counter += 1

ዋጋ B-Price

3500 I-Price

ብር I-Price

ለሱቅና O

ብዛት O

ተረካቢወች O

ባስተያየት O

እናስረክባለን O

0907000777 O

0919181353 O

አድራሻ O

አድስ B-LOC

አበባ I-LOC

ሃያሁለት I-LOC

(22 I-LOC

